In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' %x)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)


In [2]:
path = '/Users/zhuangyuhao/PycharmProjects/AC_model/'

dev = pd.read_excel(path + '结果测试_230331/会计事务所数据检查_230407.xlsx')
check = pd.read_excel(path + '整合事务所模型_20220919/tmp2/事务所表_230403.xlsx')

In [3]:
dev.drop(columns=['id', 'name', 'priority', 'describe', 'name.1', 'factor_id'], inplace=True)
dev.dropna(subset='org_name', inplace=True)
dev.rename(columns={'org_name': 'AF_id_name',
                    'code': 'indicator_code',
                    'year': 'AF_id_year',
                    'value': 'indicator_value'},
           inplace=True)

In [4]:
check.fillna(-999999.9999, inplace=True)
check.rename(columns={'AF_CYD_factor01': 'AF_id_name',
                      'AF_CYD_factor04': 'AF_id_year'},
             inplace=True)

In [5]:
check_melted = check.melt(id_vars=['AF_id_name', 'AF_id_year'],
                          value_vars=check.columns[3:],
                          var_name='indicator_code',
                          value_name='indicator_value_check')

In [6]:
check_result = dev.merge(check_melted, on=['AF_id_name', 'AF_id_year', 'indicator_code'], how='left')
check_result['indicator_value_check'] = check_result['indicator_value_check'].astype(float)
check_result['indicator_value'] = check_result['indicator_value'].astype(float)

In [7]:
check_result['match'] = np.where(check_result.indicator_value != check_result.indicator_value_check, 0, 1)

In [8]:
check_result

,AF_id_name,indicator_code,AF_id_year,indicator_value,indicator_value_check,match
0,上会会计师事务所,AF_factor01,2021,22.000,22.000,1
1,上会会计师事务所,AF_factor02,2021,445.000,445.000,1
2,上会会计师事务所,AF_factor03,2021,8.000,8.000,1
3,上会会计师事务所,AF_factor04,2021,55.000,55.000,1
4,上会会计师事务所,AF_factor05,2021,60499000.000,60499000.000,1
...,...,...,...,...,...,...
7123,鹏盛会计师事务所,AF_factor50,2019,0.000,NaN,0
7124,鹏盛会计师事务所,AF_factor50,2019,0.000,NaN,0
7125,鹏盛会计师事务所,AF_factor50,2019,0.000,NaN,0
7126,鹏盛会计师事务所,AF_factor50,2019,0.000,NaN,0


In [9]:
# 排除误差小于等于0.01的，非比值的情况
check_result['match'] = np.where(((check_result['indicator_code'] == 'AF_factor06')|(check_result['indicator_code'] == 'AF_factor07'))&(abs(check_result['indicator_value']-check_result['indicator_value_check'])<=0.01), 1, check_result['match'])

In [10]:
check_result_error = check_result[check_result['match'] == 0]
check_result_error.dropna(subset='indicator_value_check', inplace=True)
# 重新确认缺失值
check_result_error = check_result_error[check_result_error['indicator_code'] != 'AF_id_income']
check_result_error = check_result_error[check_result_error['indicator_code'] != 'AF_factor01']  # 相同
check_result_error = check_result_error[check_result_error['indicator_code'] != 'AF_factor02']  # 相同

/var/folders/21/r5tywv_91vz_8s1tgrs4qpjm0000gn/T/ipykernel_21668/841693994.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check_result_error.dropna(subset='indicator_value_check', inplace=True)


In [11]:
# 排除【前五大客户】并列导致的问题
check_result_error = check_result_error[check_result_error['indicator_code'] != 'AF_factor10']  # 自己错
check_result_error = check_result_error[check_result_error['indicator_code'] != 'AF_factor10_0']  # 自己错

In [12]:
# 检查错误
check_result_error = check_result_error[check_result_error['indicator_code'] != 'AF_factor19']  # 开发错 抽样5



In [13]:
check_result_error

,AF_id_name,indicator_code,AF_id_year,indicator_value,indicator_value_check,match
19,上会会计师事务所,AF_factor18_0,2021,42.000,1.000,0
20,上会会计师事务所,AF_factor18,2021,0.760,0.018,0
23,上会会计师事务所,AF_factor20,2021,0.080,0.081,0
25,上会会计师事务所,AF_factor21,2021,0.050,0.055,0
59,上会会计师事务所,AF_factor15,2020,0.020,0.022,0
...,...,...,...,...,...,...
6801,苏亚金诚会计师事务所,AF_factor21,2020,0.190,0.188,0
6835,苏亚金诚会计师事务所,AF_factor15,2019,0.030,0.033,0
6843,苏亚金诚会计师事务所,AF_factor20,2019,0.040,0.039,0
7015,鹏盛会计师事务所,AF_factor18_0,2021,1.000,0.000,0


In [14]:
# check_result_error.to_excel(path + '结果测试_230331/测试结果_事务所_error_230407.xlsx', index=False)